### 1. Импорт нужных библиотек

In [74]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd
from datetime import datetime

import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from catboost import CatBoostClassifier

from matplotlib import pyplot as plt

import pickle

### 2. Обзор данных

#### 2.1. Данные об отклике абонентов

In [ ]:
data = pd.read_csv('data_train.csv')

In [ ]:
data.head()

In [ ]:
data['target'].value_counts()

Есть существенный дисбаланс классов, поэтому необходимо будет делать балансировку.

Также сразу подгрузим тестовые данные, для которых необходимо получить предсказание:

In [ ]:
data_test = pd.read_csv('data_test.csv')

Посмотрим на варианты предлагаемых услуг. Как видим ниже, всего абонентам предлагалась одна из 8 услуг.

In [ ]:
data['vas_id'].nunique()

Посмотрим, сколько услуг предлагалось одному абоненту. Как видим ниже, каждому абоненту предлагалось от 1 до 3 услуг.

In [ ]:
data.groupby(['id'])['vas_id'].count().value_counts()

Посмотрим, предлагались ли какие-то услуги абоненту одновременно или между ними значительный временной интервал.

In [ ]:
data.groupby(['id', 'buy_time'])['vas_id'].count().value_counts()

То есть крайне редко предлагалось более одной услуги в один момент времени (около 0,01% случаев).

Но еще возможна ситуация, когда разница между временем предложений совсем небольшая (допустим, не более одного дня). То есть это по сути одновременно предлагаемые услуги, хотя buy_time у них не совпадает. Поищем и такие случаи.
Один день - это 86400 секунд. То есть разница между двумя временными метками должна быть не больше чем 86400.

Для каждого пользователя найдем три времени сделанных предложений: max, median и min. Поскольку предложений было не более трех, то среди значений max, median и min может быть от 1 до 3 различных значений.
Если пользователю были сделаны предложения с интервалом не больше чем 86400, то одна из разностей значений (max - median) или (median - min) должна быть не больше чем 86400 (но больше нуля).   

In [ ]:
buy_stats = (pd.DataFrame(data.groupby(['id'])['buy_time'].max() - data.groupby(['id'])['buy_time'].median())).rename(columns={'buy_time':'buy_time_diff'})

In [ ]:
buy_stats.loc[(buy_stats['buy_time_diff'] <= 86400)& (buy_stats['buy_time_diff'] > 0)]

In [ ]:
buy_stats = (pd.DataFrame(data.groupby(['id'])['buy_time'].median() - data.groupby(['id'])['buy_time'].min())).rename(columns={'buy_time':'buy_time_diff'})

In [ ]:
buy_stats.loc[(buy_stats['buy_time_diff'] <= 86400)& (buy_stats['buy_time_diff'] > 0)]

Таким образом, не было ни одного случая, когда абоненту предлагали несколько услуг с разницей менее суток. И, как было показано ранее, ничтожно мало случаев, когда несколько услуг предлагаются одновременно.

Примем это за политику компании и будем придерживаться ее и дальше при решении задачи: абонентам не будем предлагать несколько услуг одновременно или с небольшой разницей (до суток).

Посмотрим, были ли случаи, когда одному абоненту предлагали одни и те же услуги несколько раз.

In [ ]:
data.groupby(['id', 'vas_id'])['vas_id'].count().value_counts()

Как видим, в примерно 0.75% случаев одна и та же услуга предлагалась абоненту дважды. То есть, по-видимому, это допускается политикой компании в особых случаях. В таком случае, оценивая, какие услуги предложить абоненту, мы сможем предлагать и ранее предложенные услуги (в исключительных случаях, при хорошей вероятности положительного отклика).

ВЫВОДЫ.
Для дальнейшего будем считать, что: 
1. Предлагать пользователю можно только одну услугу, даже в случае хороших прогнозов по нескольким услугам.
2. Даже если пользователь ранее уже отказался от услуги, то все равно ему можно попробовать предложить эту услугу в дальнейшем при условии очень хороших прогнозов по отклику.

Мы не будем использовать эти выводы в процессе предобработки данных и при обучении моделей, но они пригодятся для формирования итоговых предложений пользователям.

#### 2.2. Профили пользователей

In [ ]:
features = dd.read_csv('features.csv', sep='\t')

In [ ]:
features.head()

Посмотрим, много ли пользователей имеет несколько профилей на разные временные метки.

In [ ]:
%time
features.groupby('id').buy_time.count().value_counts().compute()

Как видим, около 3% пользователей имеет по два профиля, а остальные - один профиль. Поскольку процент пользователей с двумя профилями достаточно большой, то при соединении датафрейма откликов с датафреймом профилей пользователей для каждого отклика необходимо будет находить ближайший по времени профиль.

### 3. Соединение таблиц профилей и откликов

Поскольку таблица features очень большая и может не поместиться в оперативной памяти, то сначала отфильтруем ее, оставив только те профили, пользователи которых присутствуют в таблице data или data_test. Это уже будет сравнительно небольшая таблица, с которой можно работать без dask.

In [ ]:
features_cut = features.loc[features['id'].isin(list(data['id'])+list(data_test['id']))]

Теперь соединим таблицу features_cut с data и data_test.

In [ ]:
%time
merged_data = pd.merge_asof(data.sort_values('buy_time'), features_cut.compute().sort_values('buy_time'), on='buy_time', by='id', direction='nearest')

In [ ]:
%time
merged_data_test = pd.merge_asof(data_test.sort_values('buy_time'), features_cut.compute().sort_values('buy_time'), on='buy_time', by='id', direction='nearest')

In [ ]:
merged_data.head()

In [ ]:
merged_data_test.head()

In [ ]:
merged_data.info(memory_usage="deep")

In [ ]:
del data
del data_test
del features_cut

Теперь таблицы готовы для решения задачи. Сохраним их в pickle.

In [ ]:
merged_data.to_pickle("merged_data.pkl")

In [ ]:
merged_data_test.to_pickle("merged_data_test.pkl")

### 4. Предобработка данных

In [6]:
data = pd.read_pickle("merged_data.pkl")

#### 4.1. Балансировка классов

In [14]:
def balance_df_by_target(df, target_name, method='over'):

    assert method in ['over', 'under', 'tomek', 'smote'], 'Неверный метод сэмплирования'
    
    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1
    if method == 'over':
        for i in range(disbalance_coeff):
            sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
            df = df.append(sample, ignore_index=True)
            
    elif method == 'under':
        df_ = df.copy()
        df = df_[df_[target_name] == minor_class_name]
        tmp = df_[df_[target_name] == major_class_name]
        df = df.append(tmp.iloc[
            np.random.randint(0, tmp.shape[0], target_counts[minor_class_name])
        ], ignore_index=True)

    elif method == 'tomek':
        from imblearn.under_sampling import TomekLinks
        tl = TomekLinks()
        X_tomek, y_tomek = tl.fit_sample(df.drop(columns=target_name), df[target_name])
        df = pd.concat([X_tomek, y_tomek], axis=1)
    
    elif method == 'smote':
        from imblearn.over_sampling import SMOTE
        smote = SMOTE()
        X_smote, y_smote = smote.fit_sample(df.drop(columns=target_name), df[target_name])
        df = pd.concat([X_smote, y_smote], axis=1)

    return df.sample(frac=1) 

In [ ]:
data_balanced = balance_df_by_target(data, 'target', method='under')
    
data_balanced['target'].value_counts()

#### 4.2. Проверка пропусков

Проверим наличие пропусков в данных. Как увидим ниже, пропуски отсутствуют.

In [ ]:
data_balanced.isna().sum().sum()

In [ ]:
data_balanced.head()

#### 4.3. Dummy-переменные

In [ ]:
set(data_balanced.dtypes)

Все признаки числовые. Преобразуем признак vas_id в тип object, так как это по сути категориальный признак.

In [ ]:
data_balanced['vas_id'] = data_balanced.astype({'vas_id': 'object'})['vas_id']

Кодируем признак vas_id (только он является нечисловым).

In [ ]:
data_dum = pd.get_dummies(data_balanced, drop_first=True)

In [ ]:
data_dum.head()

#### 4.4. Стандартизация данных

Из данных удалим атрибуты 'Unnamed: 0_x', 'id', 'Unnamed: 0_y', не несущие значимой информации. Качество моделей будем оценивать с помощью кросс-валидации, поэтому не требуется разбиение данных на train и test (valid).

In [ ]:
X = data_dum.drop(columns=['Unnamed: 0_x', 'id', 'Unnamed: 0_y', 'target'])
y = data_dum['target']

In [ ]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [ ]:
X_scaled.head()

#### 4.5. Генерация новых признаков

Большинство признаков мы никак не можем интерпретировать, поскольку данные обезличены. Но можем попробовать что-то сделать с признаком buy_time (время покупки). 
Сейчас этот признак представлен в формате timestamp, и он может быть полезен прямо в таком виде, как числовой признак (поскольку с течением времени вполне могут наблюдаться определенные тенденции действий покупателей - подключение услуги или отказ от нее, то корректно сравнивать этот признак на больше-меньше). 
Дополнительно мы также можем сгенерировать из этого признака новые признаки, такие как день недели, месяц, час дня. Вполне возможно, что в действиях покупателей наблюдается определенная цикличность: например, ночью покупатели менее лояльны и чаще отказываются от услуги, чем подключают ее. Или, допустим, люди со стандартным графиком работы 5/2 реже подключают услуги по понедельникам (потому что "понедельник - день тяжелый").

In [ ]:
data_balanced.head()

Возможные дни недели:

In [ ]:
set(datetime.fromtimestamp(x).timetuple().tm_wday for x in data_balanced['buy_time'])

То есть каждый раз новая услуга предлагалась в понедельник. Значит, нет смысла в таком признаке для обучения модели.

Возможные часы:

In [ ]:
set(datetime.fromtimestamp(x).hour for x in data_balanced['buy_time'])

То есть новая услуга всегда предлагалась в районе полуночи, и пользы от такого признака тоже нет.

Возможные месяцы:

In [ ]:
set(datetime.fromtimestamp(x).month for x in data_balanced['buy_time'])

Как видим, в данных train использованы данные с июля по декабрь. Данный признак не будет нам полезен в ближайшей перспективе (для предсказания ответов с начала года).

Таким образом, рассмотренные признаки не являются полезными, и мы не будем их генерировать.

### 5. Baseline-решение - логистическая регрессия

В качестве базовой модели будем использовать логистическую регрессию. Для оценки качества будет использоваться кросс-валидация.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
c_values = np.logspace(-2, 3, 50)
model_lr = LogisticRegressionCV(random_state=0, Cs=c_values, cv=skf, verbose=1, n_jobs=-1, scoring='f1_macro')
model_lr.fit(X_scaled, y)

In [ ]:
pred_lr = model_lr.predict(X_scaled)

In [ ]:
f1_score(y, pred_lr, average="macro")

In [ ]:
print(classification_report(y, pred_lr))

In [ ]:
unique, counts = np.unique(pred_lr, return_counts=True)
dict(zip(unique, counts))

Таким образом, базовое решение дало f1-score 0,671. Будем пробовать его улучшать.

### 6. Другие алгоритмы

Ранее выполненная предобработка данных подойдет и для следующих алгоритмов, которые мы попробуем применить: решающие деревья и CatBoost.

#### 6.1. Решающие деревья

In [ ]:
parameters = {
    'max_features': np.arange(5, 10),
    'max_depth': np.arange(5, 10),
}
            

clf = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=parameters,
    scoring='f1_macro',
    cv=5,
)

In [ ]:
%time
clf.fit(X_scaled, y)

In [ ]:
pred_clf = clf.predict(X_scaled)

In [ ]:
f1_score(y, pred_clf, average="macro")

In [ ]:
print(classification_report(y, pred_clf))

Полученный результат хуже результата базового решения, на основе логистической регрессии.

#### 6.2. Catboost

In [ ]:
model_catb = CatBoostClassifier()

In [ ]:
parameters = {'depth' : [4,5,6,7,8,9,10],
              'learning_rate' : [0.01,0.02,0.03,0.04],
              'iterations' : [10, 20]
                 }

In [ ]:
frozen_params = {
     'silent':True,
     'random_state':21,
     'eval_metric':'F1',
     'early_stopping_rounds':20
}

In [ ]:
grid_model_catb = GridSearchCV(estimator=model_catb, param_grid = parameters, cv = 5, scoring='f1_macro')
grid_model_catb.fit(X_scaled, y)

In [ ]:
pred_catb = grid_model_catb.predict(X_scaled)

In [ ]:
f1_score(y, pred_catb, average="macro")

In [ ]:
grid_model_catb.best_params_

In [ ]:
print(classification_report(y, pred_catb))

Таким образом, f1_score равен 0,888 - выше, чем в предыдущих моделях.

### 7. Снижение размерности пространства признаков

Посмотрим, сможем ли мы улучшить результат работы CatBoost, если предварительно выполним снижение размерности пространства признаков с помощью метода PCA.

Оценим, сколько необходимо взять главных компонент. Для этого посмотрим на накапливаемую сумму параметра explained_variance_ratio_.

In [ ]:
pca = PCA().fit(X_scaled)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

Подберем параметр n_components в PCA так, чтобы процент объясненной дисперсии был не менее чем 95%.

In [ ]:
cumsum_variance = np.cumsum(pca.explained_variance_ratio_)
for i in range(len(cumsum_variance)):
    if cumsum_variance[i] >=0.95:
        break
print(i)

Как видим, можно взять n_components равный 151.

In [ ]:
pca_2 = PCA(n_components=145)
principalComponents_2 = pca.fit_transform(X_scaled)
principal_data_2 = pd.DataFrame(data = principalComponents)

In [ ]:
principal_data_2.head()

Теперь можно пробовать вновь применять метод CatBoost.

In [ ]:
model_catb_2 = CatBoostClassifier()

In [ ]:
parameters = {'depth' : [4,5,6,7,8,9,10],
              'learning_rate' : [0.01,0.02,0.03,0.04],
              'iterations' : [10, 20]
                 }

In [ ]:
frozen_params = {
     'silent':True,
     'random_state':21,
     'eval_metric':'F1',
     'early_stopping_rounds':20
}

In [ ]:
grid_model_catb_2 = GridSearchCV(estimator=model_catb_2, param_grid = parameters, cv = 5, scoring='f1_macro')
grid_model_catb_2.fit(principal_data, y)

In [ ]:
pred_catb_2 = grid_model_catb_2.predict(principal_data_2)

In [ ]:
f1_score(y, pred_catb_2, average="macro")

In [ ]:
grid_model_catb_2.best_params_

In [ ]:
print(classification_report(y, pred_catb_2))

Как видим, качество работы модели ухудшилось.

Попробуем понизить размерность не настолько сильно. Ранее использовали параметр n_components равный 145, а теперь возьмем его равным 180.

In [ ]:
pca_3 = PCA(n_components=180)
principalComponents_3 = pca.fit_transform(X_scaled)
principal_data_3 = pd.DataFrame(data = principalComponents)

In [ ]:
principal_data_3.head()

In [ ]:
model_catb_3 = CatBoostClassifier()

In [ ]:
parameters = {'depth' : [4,5,6,7,8,9,10],
              'learning_rate' : [0.01,0.02,0.03,0.04],
              'iterations' : [10, 20]
                 }

In [ ]:
frozen_params = {
     'silent':True,
     'random_state':21,
     'eval_metric':'F1',
     'early_stopping_rounds':20
}

In [ ]:
grid_model_catb_3 = GridSearchCV(estimator=model_catb_3, param_grid = parameters, cv = 5, scoring='f1_macro')
grid_model_catb_3.fit(principal_data_3, y)

In [ ]:
pred_catb_3 = grid_model_catb_3.predict(principal_data_3)

In [ ]:
f1_score(y, pred_catb_3, average="macro")

In [ ]:
grid_model_catb_3.best_params_

In [ ]:
print(classification_report(y, pred_catb_3))

f1_score вырос, но незначительно (с 0.821 до 0.824). По-видимому, при понижении размерности мы теряем довольно много информации, существенной для решения задачи классификации. Поэтому не будем применять этот метод в конечном решении.

Итак, для финального решения будем использовать CatBoost с параметрами grid_model_catb.best_params_ ('depth': 4, 'iterations': 10, 'learning_rate': 0.03).

### 8. Прогнозирование на тестовом датасете

In [ ]:
pred_test = grid_model_catb.predict(data_test)

In [ ]:
pred_test_proba = grid_model_catb.predict_proba(data_test)

In [ ]:
pred_test_proba

In [ ]:
pred_test

In [ ]:
test = pd.read_csv("data_test".csv)

In [ ]:
test['target'] = pred_test

### 9. Создание и сохранение пайплайна

Итоговый пайплайн должен выполнять следующие последовательные действия:
   1. Готовить исходные данные: 
      - соединять таблицы откликов и профилей;
      - создавать dummy-переменные;
      - выполнять стандартизацию данных;
   2. Выполнять классификацию с помощью обученной модели CatBoost.

При обучении модели будем подавать на вход таблицу, в которой уже произведена балансировка классов.

#### 9.1. Создание пайплайна

In [26]:
# Трансформер для соединения таблиц откликов и профилей
class JoinTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.features = dd.read_csv('features.csv', sep='\t')

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        features_cut = self.features.loc[self.features['id'].isin(X['id'])]
        merged_data = pd.merge_asof(X.sort_values('buy_time'), features_cut.compute().sort_values('buy_time'), on='buy_time', by='id', direction='nearest')
        return merged_data

In [27]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key, drop_first=True)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [28]:
categorical_columns = ['vas_id']
numerical_columns = list(data.columns)
numerical_columns.remove('Unnamed: 0_x')
numerical_columns.remove('id')
numerical_columns.remove('Unnamed: 0_y')
numerical_columns.remove('target')
numerical_columns.remove('vas_id')

Сделаем отдельно два пайплайна: один для соединения таблиц data и features, другой для предобработки данных и обучения модели.

In [48]:
pipeline_merge = Pipeline([('join', JoinTransformer())])

In [49]:
final_transformers = []

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for num_col in numerical_columns:
    num_transformer = Pipeline([
                ('selector', NumberSelector(key=num_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((num_col, num_transformer))

In [50]:
feats = FeatureUnion(final_transformers)

In [53]:
pipeline_catboost = Pipeline([
    ('features', feats),
    ('classifier', CatBoostClassifier(depth=4, iterations=10, learning_rate=0.03, silent=True, random_state=21,
                                eval_metric='F1', early_stopping_rounds=20)),
])

#### 9.2. Обучение модели

In [54]:
data_train = pd.read_csv('data_train.csv')

In [55]:
data_train_balanced = balance_df_by_target(data, 'target', method='under')
    
data_train_balanced['target'].value_counts()

C:\Users\Diman\AppData\Local\Temp\ipykernel_3080\2281545907.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp.iloc[


0.0    60186
1.0    60186
Name: target, dtype: int64

In [56]:
X_train = data_train_balanced.drop(columns=['target'])
y_train = data_train_balanced['target']

In [57]:
pipeline_merge.fit(X_train, y_train)

Pipeline(steps=[('join', JoinTransformer())])

In [58]:
X_train_merged = pipeline_merge.transform(X_train)

In [59]:
pipeline_catboost.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('vas_id',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='vas_id')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='vas_id'))])),
                                                ('buy_time',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='buy_time')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('0',
                                                 Pipeline(steps=[('selector',
                              

#### 9.3. Предсказание отклика

In [ ]:
data_test = pd.read_csv('data_test.csv')

In [60]:
data_test_merged = pipeline_merge.transform(data_test)

In [65]:
preds = pipeline_catboost.predict_proba(data_test_merged)[:, 1]
preds[:10]

array([0.31640558, 0.31640558, 0.64402677, 0.64402677, 0.64402677,
       0.64402677, 0.64402677, 0.31640558, 0.31640558, 0.59706428])

In [67]:
data_test['target'] = preds

In [69]:
data_test.to_csv('answers_test.csv')

#### 9.4. Сохранение моделей

In [77]:
file = open('pipeline_merge.pkl', 'wb')

In [78]:
pickle.dump(pipeline_merge, file)

In [79]:
file.close()

In [80]:
file = open('pipeline_catboost.pkl', 'wb')

In [81]:
pickle.dump(pipeline_catboost, file)

In [82]:
file.close()

### 10. Применение моделей

Теперь попробуем загрузить и применить модели для предсказания отклика.

In [88]:
data_test = pd.read_csv('data_test.csv')

In [86]:
with open('pipeline_merge.pkl', 'rb') as f:
        pipeline_merge_test = pickle.load(f)

In [87]:
with open('pipeline_catboost.pkl', 'rb') as f:
        pipeline_catboost_test = pickle.load(f)

In [89]:
data_test_test = pipeline_merge_test.transform(data_test)

In [90]:
preds = pipeline_catboost_test.predict_proba(data_test_test)[:, 1]

In [96]:
data_test['target'] = preds

In [95]:
data_test.head()

,Unnamed: 0,id,vas_id,buy_time,target
0,0,3130519,2.0,1548018000,0.316406
1,1,2000860,4.0,1548018000,0.316406
2,2,1099444,2.0,1546808400,0.644027
3,3,1343255,5.0,1547413200,0.644027
4,4,1277040,2.0,1546808400,0.644027


In [97]:
data_test.to_csv('answers_test.csv')

Работает!